# Lab 2: Building and Deploying Gradio Demos

In this lab, we will build interactive Gradio demos for our food classification models and deploy them to Hugging Face Spaces.

## Learning Objectives

By the end of this lab, you will be able to:
- Build interactive web demos using Gradio
- Structure a deployable ML application
- Deploy models to Hugging Face Spaces
- Scale up to larger datasets (Food101) with Food Classifier Big

## 0. Setup

First, let's set up our environment and install Gradio.

In [24]:
%pip install torch torchvision torchinfo gradio
import torch
from torch import nn
import torchvision
from torchvision import transforms
from pathlib import Path
import gradio as gr

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Note: you may need to restart the kernel to use updated packages.
Using device: cpu



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
import requests
import zipfile

data_path = Path('data/')
image_path = data_path / 'hamburger_hot_dog_french_fries'

if image_path.is_dir() and (image_path / 'train').is_dir():
    print(f'{image_path} directory exists.')
else:
    print(f'Setting up {image_path} directory...')
    data_path.mkdir(parents=True, exist_ok=True)
    
    zip_file_path = data_path / 'hamburger_hot_dog_french_fries.zip'
    with open(zip_file_path, 'wb') as f:
        request = requests.get('https://github.com/poridhioss/Introduction-to-Deep-Learning-with-Pytorch-Resources/raw/refs/heads/main/Model-deployment/hamburger_hot_dog_french_fries.zip')
        print('Downloading data...')
        f.write(request.content)
    
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        print('Unzipping data...')
        # Extract to data_path since zip already contains the hamburger_hot_dog_french_fries folder
        zip_ref.extractall(data_path)
    print('Done!')

train_dir = image_path / 'train'
test_dir = image_path / 'test'
print(f'Train directory: {train_dir}')
print(f'Test directory: {test_dir}')

data\hamburger_hot_dog_french_fries directory exists.
Train directory: data\hamburger_hot_dog_french_fries\train
Test directory: data\hamburger_hot_dog_french_fries\test


In [26]:
from timeit import default_timer as timer

def create_effnetb2_model(num_classes: int = 3, seed: int = 42):
    weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    transforms = weights.transforms()
    model = torchvision.models.efficientnet_b2(weights=weights)
    
    # Freeze base layers
    for param in model.parameters():
        param.requires_grad = False
    
    torch.manual_seed(seed)
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.3, inplace=True),
        nn.Linear(in_features=1408, out_features=num_classes)
    )
    
    return model, transforms

# Note: ImageFolder loads classes in alphabetical order
class_names = ["french_fries", "hamburger", "hot_dog"]

effnetb2, effnetb2_transforms = create_effnetb2_model(num_classes=3)
print('EffNetB2 model created!')
print(f'Class names: {class_names}')

EffNetB2 model created!
Class names: ['french_fries', 'hamburger', 'hot_dog']


In [27]:
from typing import Dict, List, Tuple
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import os

def create_dataloaders(
    train_dir: str, 
    test_dir: str, 
    transform: transforms.Compose, 
    batch_size: int, 
    num_workers: int = os.cpu_count()
) -> Tuple[DataLoader, DataLoader, List[str]]:
    from torchvision import datasets
    
    train_data = datasets.ImageFolder(train_dir, transform=transform)
    test_data = datasets.ImageFolder(test_dir, transform=transform)
    class_names = train_data.classes

    train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
    test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

    return train_dataloader, test_dataloader, class_names

def train_step(model, dataloader, loss_fn, optimizer, device):
    model.train()
    train_loss, train_acc = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item() / len(y_pred)
    return train_loss / len(dataloader), train_acc / len(dataloader)

def test_step(model, dataloader, loss_fn, device):
    model.eval()
    test_loss, test_acc = 0, 0
    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            test_pred_logits = model(X)
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item() / len(test_pred_labels))
    return test_loss / len(dataloader), test_acc / len(dataloader)

def train(model, train_dataloader, test_dataloader, optimizer, loss_fn, epochs, device):
    results = {"train_loss": [], "train_acc": [], "test_loss": [], "test_acc": []}
    model.to(device)
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model, train_dataloader, loss_fn, optimizer, device)
        test_loss, test_acc = test_step(model, test_dataloader, loss_fn, device)
        print(f"Epoch: {epoch+1} | train_loss: {train_loss:.4f} | train_acc: {train_acc:.4f} | test_loss: {test_loss:.4f} | test_acc: {test_acc:.4f}")
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)
    return results

def save_model(model, target_dir, model_name):
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True, exist_ok=True)
    model_save_path = target_dir_path / model_name
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(), f=model_save_path)

def plot_loss_curves(results: Dict[str, List[float]]):
    import matplotlib.pyplot as plt
    loss = results["train_loss"]
    test_loss = results["test_loss"]
    accuracy = results["train_acc"]
    test_accuracy = results["test_acc"]
    epochs = range(len(results["train_loss"]))
    plt.figure(figsize=(15, 7))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, label="train_loss")
    plt.plot(epochs, test_loss, label="test_loss")
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, label="train_accuracy")
    plt.plot(epochs, test_accuracy, label="test_accuracy")
    plt.title("Accuracy")
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()

print("[INFO] Helper functions defined successfully!")

[INFO] Helper functions defined successfully!


In [28]:
print("[INFO] Creating DataLoaders...")
train_dataloader, test_dataloader, class_names = create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=effnetb2_transforms,
    batch_size=32
)

print(f"[INFO] Class names: {class_names}")
print(f"[INFO] Training samples: {len(train_dataloader.dataset)}")
print(f"[INFO] Testing samples: {len(test_dataloader.dataset)}")

optimizer = torch.optim.Adam(params=effnetb2.parameters(), lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()

print("\n[INFO] Training model...")
effnetb2_results = train(
    model=effnetb2,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    epochs=5,
    device=device
)

save_model(
    model=effnetb2,
    target_dir="models",
    model_name="pretrained_effnetb2_feature_extractor_hamburger_hot_dog_french_fries.pth"
)

print("\n[INFO] Training complete!")

[INFO] Creating DataLoaders...
[INFO] Class names: ['french_fries', 'hamburger', 'hot_dog']
[INFO] Training samples: 2250
[INFO] Testing samples: 750

[INFO] Training model...


  0%|          | 0/5 [00:00<?, ?it/s]f:\poridhi-ai-labs\venv\Lib\site-packages\torch\utils\data\dataloader.py:1118: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)
 20%|██        | 1/5 [05:33<22:15, 333.76s/it]

Epoch: 1 | train_loss: 0.6655 | train_acc: 0.7670 | test_loss: 0.3916 | test_acc: 0.8973


 40%|████      | 2/5 [10:48<16:06, 322.28s/it]

Epoch: 2 | train_loss: 0.4185 | train_acc: 0.8691 | test_loss: 0.3178 | test_acc: 0.9051


...

Epoch: 3 | train_loss: 0.3620 | train_acc: 0.8739 | test_loss: 0.2925 | test_acc: 0.9129


 80%|████████  | 4/5 [21:39<05:25, 325.52s/it]

Epoch: 4 | train_loss: 0.3168 | train_acc: 0.8934 | test_loss: 0.2715 | test_acc: 0.9129


100%|██████████| 5/5 [26:56<00:00, 323.21s/it]

Epoch: 5 | train_loss: 0.3157 | train_acc: 0.8864 | test_loss: 0.2704 | test_acc: 0.9185
[INFO] Saving model to: models\pretrained_effnetb2_feature_extractor_hamburger_hot_dog_french_fries.pth

[INFO] Training complete!


## 7. Bringing Food Classifier to life by creating a Gradio demo

We've decided we'd like to deploy the EffNetB2 model (to begin with, this could always be changed later).

So how can we do that?

There are several ways to deploy a machine learning model each with specific use cases (as discussed above).

We're going to be focused on perhaps the quickest and certainly one of the most fun ways to get a model deployed to the internet.

And that's by using [Gradio](https://gradio.app/).

Why create a demo of your models?

Because metrics on the test set look nice but you never really know how your model performs until you use it in the wild.

![Gradio Workflow](https://raw.githubusercontent.com/poridhiEng/lab-asset/cdf9e5ca3eda11102b5f8493572eb84e83523d47/tensorcode/Deep-learning-with-pytorch/Model-Deployment/Lab_02/images/img4.svg)

So let's get deploying!

We'll start by importing Gradio with the common alias `gr` and if it's not present, we'll install it.

In [29]:
print(f"Gradio version: {gr.__version__}")

Gradio version: 6.3.0


Gradio ready!

Let's turn Food Classifier into a demo application.

### 7.1 Creating a function to map our inputs and outputs

To create our Food Classifier demo with Gradio, we'll need a function to map our inputs to our outputs.

We created a function earlier called `pred_and_store()` to make predictions with a given model across a list of target files and store them in a list of dictionaries.

How about we create a similar function but this time focusing on making a prediction on a single image with our EffNetB2 model?

More specifically, we want a function that takes an image as input, preprocesses (transforms) it, makes a prediction with EffNetB2 and then returns the prediction (pred or pred label for short) as well as the prediction probability (pred prob).

And while we're here, let's return the time it took to do so too:

```
input: image -> transform -> predict with EffNetB2 -> output: pred, pred prob, time taken
```

This will be our `fn` parameter for our Gradio interface.

First, let's make sure our EffNetB2 model is on the CPU (since we're sticking with CPU-only predictions, however you could change this if you have access to a GPU).

In [30]:
effnetb2.to("cpu") 

next(iter(effnetb2.parameters())).device

device(type='cpu')

And now let's create a function called `predict()` to replicate the workflow above.

In [31]:
from typing import Tuple, Dict

def predict(img) -> Tuple[Dict, float]:

    start_time = timer()
    
    # Transform the target image and add a batch dimension
    img = effnetb2_transforms(img).unsqueeze(0)
    
    # Put model into evaluation mode and turn on inference mode
    effnetb2.eval()
    with torch.inference_mode():
        # Pass the transformed image through the model and turn the prediction logits into prediction probabilities
        pred_probs = torch.softmax(effnetb2(img), dim=1)
    
    # Create a prediction label and prediction probability dictionary for each prediction class (this is the required format for Gradio's output parameter)
    pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}
    
    pred_time = round(timer() - start_time, 5)
    
    return pred_labels_and_probs, pred_time

Beautiful! 

Now let's see our function in action by performing a prediction on a random image from the test dataset.

We'll start by getting a list of all the image paths from the test directory and then randomly selecting one.

Then we'll open the randomly selected image with `PIL.Image.open()`.

Finally, we'll pass the image to our `predict()` function.

In [32]:
import random
from PIL import Image

test_data_paths = list(Path(test_dir).glob("*/*.jpg"))

random_image_path = random.sample(test_data_paths, k=1)[0]

image = Image.open(random_image_path)
print(f"[INFO] Predicting on image at path: {random_image_path}\n")

pred_dict, pred_time = predict(img=image)
print(f"Prediction label and probability dictionary: \n{pred_dict}")
print(f"Prediction time: {pred_time} seconds")

[INFO] Predicting on image at path: data\hamburger_hot_dog_french_fries\test\hot_dog\156135.jpg

Prediction label and probability dictionary: 
{'french_fries': 0.0037253673654049635, 'hamburger': 0.03534900024533272, 'hot_dog': 0.9609256386756897}
Prediction time: 0.17936 seconds


Nice!

Running the cell above a few times we can see different prediction probabilities for each label from our EffNetB2 model as well as the time it took per prediction.

### 7.3 Creating a list of example images

Our `predict()` function enables us to go from inputs -> transform -> ML model -> outputs.

Which is exactly what we need for our Graido demo.

But before we create the demo, let's create one more thing: a list of examples.

Gradio's `Interface` class takes a list of `examples` of as an optional parameter (`gradio.Interface(examples=List[Any])`).

And the format for the `examples` parameter is a list of lists.

So let's create a list of lists containing random filepaths to our test images.

Three examples should be enough.

In [33]:
example_list = [[str(filepath)] for filepath in random.sample(test_data_paths, k=3)]
example_list

[['data\\hamburger_hot_dog_french_fries\\test\\hamburger\\1960715.jpg'],
 ['data\\hamburger_hot_dog_french_fries\\test\\hamburger\\2007244.jpg'],
 ['data\\hamburger_hot_dog_french_fries\\test\\french_fries\\2823700.jpg']]

Perfect!

Our Gradio demo will showcase these as example inputs to our demo so people can try it out and see what it does without uploading any of their own data. 

### 7.4 Building a Gradio interface

Time to put everything together and bring our Food Classifier demo to life!

Let's create a Gradio interface to replicate the workflow:

```
input: image -> transform -> predict with EffNetB2 -> output: pred, pred prob, time taken
```

We can do with the `gradio.Interface()` class with the following parameters:
* `fn` - a Python function to map `inputs` to `outputs`, in our case, we'll use our `predict()` function.
* `inputs` - the input to our interface, such as an image using `gradio.Image()` or `"image"`. 
* `outputs` - the output of our interface once the `inputs` have gone through the `fn`, such as a label using `gradio.Label()` (for our model's predicted labels) or number using `gradio.Number()` (for our model's prediction time).
    * **Note:** Gradio comes with many in-built `inputs` and `outputs` options known as "Components".
* `examples` - a list of examples to showcase for the demo.
* `title` - a string title of the demo.
* `description` - a string description of the demo.
* `article` - a reference note at the bottom of the demo.

Once we've created our demo instance of `gr.Interface()`, we can bring it to life using `gradio.Interface().launch()` or `demo.launch()` command. 

Easy!

In [34]:
import gradio as gr

title = "Food Classifier 🍔🌭🍟"
description = "An EfficientNetB2 feature extractor computer vision model to classify images of food as hamburger, hot dog or french fries."
article = "Created at [PyTorch Model Deployment](https://www.learnpytorch.io/pytorch_model_deployment/)."

demo = gr.Interface(fn=predict, # mapping function from input to output
                    inputs=gr.Image(type="pil"), # what are the inputs?
                    outputs=[gr.Label(num_top_classes=3, label="Predictions"), # what are the outputs?
                             gr.Number(label="Prediction time (s)")], # our fn has two outputs, therefore we have two outputs
                    examples=example_list, 
                    title=title,
                    description=description,
                    article=article)

demo.launch(debug=False, # print errors locally?
            share=True) # generate a publically shareable URL?

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://ed9bf612f4564bd301.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Woohoo!!! What an epic demo!!!

Food Classifier has officially come to life in an interface someone could use and try out.

## 8. Turning our Food Classifier Gradio Demo into a deployable app

We've seen our Food Classifier model come to life through a Gradio demo.

But what if we wanted to share it with our friends?

Well, we could use the provided Gradio link, however, the shared link only lasts for 72-hours.

To make our Food Classifier demo more permanent, we can package it into an app and upload it to [Hugging Face Spaces](https://huggingface.co/spaces/launch).

### 8.2 Deployed Gradio app structure

To upload our demo Gradio app, we'll want to put everything relating to it into a single directory.

For example, our demo might live at the path `demos/food_classifier/` with the file structure:

```
demos/
└── food_classifier/
    ├── pretrained_effnetb2_feature_extractor_hamburger_hot_dog_french_fries.pth
    ├── app.py
    ├── examples/
    │   ├── example_1.jpg
    │   ├── example_2.jpg
    │   └── example_3.jpg
    ├── model.py
    └── requirements.txt
```

Where:
* `pretrained_effnetb2_feature_extractor_hamburger_hot_dog_french_fries.pth` is our trained PyTorch model file.
* `app.py` contains our Gradio app (similar to the code that launched the app).
    * **Note:** `app.py` is the default filename used for Hugging Face Spaces, if you deploy your app there, Spaces will by default look for a file called `app.py` to run. This is changeable in settings.
* `examples/` contains example images to use with our Gradio app.
* `model.py` contains the model definition as well as any transforms associated with the model.
* `requirements.txt` contains the dependencies to run our app such as `torch`, `torchvision` and `gradio`.

Why this way?

Because it's one of the simplest layouts we could begin with. 

Our focus is: *experiment, experiment, experiment!* 

The quicker we can run smaller experiments, the better our bigger ones will be.

Here's a visual representation of the deployment structure:

![Deployment Structure](https://raw.githubusercontent.com/poridhiEng/lab-asset/cdf9e5ca3eda11102b5f8493572eb84e83523d47/tensorcode/Deep-learning-with-pytorch/Model-Deployment/Lab_02/images/img2.svg)

<!-- We're going to work towards recreating the structure above but you can see a live demo app running on Hugging Face Spaces as well as the file structure:
* [Live Gradio demo of Food Classifier 🍔🌭🍟](https://huggingface.co/spaces/mrdbourke/food_classifier).
* [Food Classifier file structure on Hugging Face Spaces](https://huggingface.co/spaces/mrdbourke/food_classifier/tree/main). -->

### 8.3 Creating a `demos` folder to store our Food Classifier app files

To begin, let's first create a `demos/` directory to store all of our Food Classifier app files.

We can do with Python's `pathlib.Path("path_to_dir")` to establish the directory path and `pathlib.Path("path_to_dir").mkdir()` to create it. 

In [35]:
import shutil
from pathlib import Path

food_classifier_demo_path = Path("demos/food_classifier/")

if food_classifier_demo_path.exists():
    shutil.rmtree(food_classifier_demo_path)
food_classifier_demo_path.mkdir(parents=True, 
                                exist_ok=True)
    
import os
print(os.listdir("demos/food_classifier/"))

[]


### 8.4 Creating a folder of example images to use with our Food Classifier demo

Now we've got a directory to store our Food Classifier demo files, let's add some examples to it.

Three example images from the test dataset should be enough.

To do so we'll:
1. Create an `examples/` directory within the `demos/food_classifier` directory.
2. Choose three random images from the test dataset and collect their filepaths in a list.
3. Copy the three random images from the test dataset to the `demos/food_classifier/examples/` directory.

In [36]:
import shutil
from pathlib import Path

food_classifier_examples_path = food_classifier_demo_path / "examples"
food_classifier_examples_path.mkdir(parents=True, exist_ok=True)

test_images = list(Path(test_dir).glob("*/*.jpg"))
if len(test_images) >= 3:
    food_classifier_examples = test_images[:3]
else:
    print(f"[WARNING] Only found {len(test_images)} test images")
    food_classifier_examples = test_images

for example in food_classifier_examples:
    destination = food_classifier_examples_path / example.name
    print(f"[INFO] Copying {example} to {destination}")
    shutil.copy2(src=example, dst=destination)

[INFO] Copying data\hamburger_hot_dog_french_fries\test\french_fries\1008163.jpg to demos\food_classifier\examples\1008163.jpg
[INFO] Copying data\hamburger_hot_dog_french_fries\test\french_fries\1033213.jpg to demos\food_classifier\examples\1033213.jpg
[INFO] Copying data\hamburger_hot_dog_french_fries\test\french_fries\10500.jpg to demos\food_classifier\examples\10500.jpg


Now to verify our examples are present, let's list the contents of our `demos/food_classifier/examples/` directory with `os.listdir()` and then format the filepaths into a list of lists (so it's compatible with Gradio's `gradio.Interface()` `example` parameter).

In [37]:
import os

example_list = [["examples/" + example] for example in os.listdir(food_classifier_examples_path)]
example_list

[['examples/1008163.jpg'], ['examples/1033213.jpg'], ['examples/10500.jpg']]

### 8.5 Moving our trained EffNetB2 model to our Food Classifier demo directory

We previously saved our Food Classifier EffNetB2 feature extractor model under `models/pretrained_effnetb2_feature_extractor_hamburger_hot_dog_french_fries.pth`.

And rather double up on saved model files, let's move our model to our `demos/food_classifier` directory.

We can do so using Python's `shutil.copy2()` method and passing in `src` (the source path of the target file) and `dst` (the destination path of the target file to be copied to) parameters.

In [38]:
import shutil

effnetb2_food_classifier_model_path = Path("models/pretrained_effnetb2_feature_extractor_hamburger_hot_dog_french_fries.pth")

effnetb2_food_classifier_model_destination = food_classifier_demo_path / effnetb2_food_classifier_model_path.name

try:
    print(f"[INFO] Attempting to copy {effnetb2_food_classifier_model_path} to {effnetb2_food_classifier_model_destination}")
    
    shutil.copy2(src=effnetb2_food_classifier_model_path, 
                 dst=effnetb2_food_classifier_model_destination)
    
    print(f"[INFO] Model copy complete.")

except FileNotFoundError:
    print(f"[INFO] No model found at {effnetb2_food_classifier_model_path}")
    print(f"[INFO] Please make sure you've run the training cells above first.")
except Exception as e:
    print(f"[INFO] Error: {e}")
    print(f"[INFO] Model exists at {effnetb2_food_classifier_model_destination}: {effnetb2_food_classifier_model_destination.exists()}")

[INFO] Attempting to copy models\pretrained_effnetb2_feature_extractor_hamburger_hot_dog_french_fries.pth to demos\food_classifier\pretrained_effnetb2_feature_extractor_hamburger_hot_dog_french_fries.pth
[INFO] Model copy complete.


### 8.6 Turning our EffNetB2 model into a Python script (`model.py`)

Our current model's `state_dict` is saved to `demos/food_classifier/pretrained_effnetb2_feature_extractor_hamburger_hot_dog_french_fries.pth`.

To load it in we can use `model.load_state_dict()` along with `torch.load()`.


But before we can do this, we first need a way to instantiate a `model`.

To do this in a modular fashion we'll create a script called `model.py` which contains our `create_effnetb2_model()` function.

That way we can import the function in *another* script (see `app.py` below) and then use it to create our EffNetB2 `model` instance as well as get its appropriate transforms.

Here, we'll use the `%%writefile path/to/file` magic command to turn a cell of code into a file.

In [39]:
%%writefile demos/food_classifier/model.py
import torch
import torchvision

from torch import nn


def create_effnetb2_model(num_classes:int=3, seed:int=42):

    weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    transforms = weights.transforms()
    model = torchvision.models.efficientnet_b2(weights=weights)

    # Freeze all layers in base model
    for param in model.parameters():
        param.requires_grad = False

    torch.manual_seed(seed)
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.3, inplace=True),
        nn.Linear(in_features=1408, out_features=num_classes),
    )
    
    return model, transforms

Writing demos/food_classifier/model.py


### 8.7 Turning our Food Classifier Gradio app into a Python script (`app.py`)

We've now got a `model.py` script as well as a path to a saved model `state_dict` that we can load in.

Time to construct `app.py`.

We call it `app.py` because by default when you create a HuggingFace Space, it looks for a file called `app.py` to run and host (though you can change this in settings).

In [40]:
%%writefile demos/food_classifier/app.py
import gradio as gr
import os
import torch

from model import create_effnetb2_model
from timeit import default_timer as timer
from typing import Tuple, Dict

class_names = ["french_fries", "hamburger", "hot_dog"]

effnetb2, effnetb2_transforms = create_effnetb2_model(
    num_classes=3, 
)

effnetb2.load_state_dict(
    torch.load(
        f="pretrained_effnetb2_feature_extractor_hamburger_hot_dog_french_fries.pth",
        map_location=torch.device("cpu"), 
    )
)

def predict(img) -> Tuple[Dict, float]:

    start_time = timer()
    
    img = effnetb2_transforms(img).unsqueeze(0)
    effnetb2.eval()
    with torch.inference_mode():
        pred_probs = torch.softmax(effnetb2(img), dim=1)
    
    pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}
    
    pred_time = round(timer() - start_time, 5)
    
    return pred_labels_and_probs, pred_time

title = "Food Classifier 🍔🌭🍟"
description = "An EfficientNetB2 feature extractor computer vision model to classify images of food as hamburger, hot dog or french fries."
article = "Created at PyTorch Model Deployment."

example_list = [["examples/" + example] for example in os.listdir("examples")]

demo = gr.Interface(fn=predict, 
                    inputs=gr.Image(type="pil"),
                    outputs=[gr.Label(num_top_classes=3, label="Predictions"), 
                             gr.Number(label="Prediction time (s)")], 
                    examples=example_list, 
                    title=title,
                    description=description,
                    article=article)

demo.launch()

Writing demos/food_classifier/app.py


### 8.8 Creating a requirements file for Food Classifier (`requirements.txt`)


In [41]:
%%writefile demos/food_classifier/requirements.txt
torch>=2.0.0
torchvision>=0.15.0
gradio>=4.0.0

Writing demos/food_classifier/requirements.txt


Nice!

We've officially got all the files we need to deploy our Food Classifier demo!

## 9. Deploying our Food Classifier app to HuggingFace Spaces

We've got a file containing our Food Classifier demo, now how do we get it to run on Hugging Face Spaces?

There are two main options for uploading to a Hugging Face Space (also called a [Hugging Face Repository](https://huggingface.co/docs/hub/repositories-getting-started#getting-started-with-repositories), similar to a git repository): 
1. [Uploading via the Hugging Face Web interface (easiest)](https://huggingface.co/docs/hub/repositories-getting-started#adding-files-to-a-repository-web-ui).
2. [Uploading via the command line or terminal](https://huggingface.co/docs/hub/repositories-getting-started#terminal).
    * **Bonus:** You can also use the [`huggingface_hub` library](https://huggingface.co/docs/huggingface_hub/index) to interact with Hugging Face, this would be a good extension to the above two options.


### 9.1 Downloading our Food Classifier app files



To begin uploading our files to Hugging Face, let's now download them from Google Colab (or wherever you're running this notebook).

To do so, we'll first compress the files into a single zip folder via the command: 

```
zip -r ../food_classifier.zip * -x "*.pyc" "*.ipynb" "*__pycache__*" "*ipynb_checkpoints*"
```

Where: 
* `zip` stands for "zip" as in "please zip together the files in the following directory". 
* `-r` stands for "recursive" as in, "go through all of the files in the target directory".
* `../food_classifier.zip` is the target directory we'd like our files to be zipped to.
* `*` stands for "all the files in the current directory".
* `-x` stands for "exclude these files". 

We can download our zip file from Google Colab using [`google.colab.files.download("demos/food_classifier.zip")`](https://colab.research.google.com/notebooks/io.ipynb) (we'll put this inside a `try` and `except` block just in case we're not running the code inside Google Colab, and if so we'll print a message saying to manually download the files).

Let's try it out!

In [42]:
import zipfile
import os
from pathlib import Path

# Create zip file using Python (cross-platform)
zip_path = Path("demos/food_classifier.zip")
source_dir = Path("demos/food_classifier")

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(source_dir):
        # Skip unwanted directories
        dirs[:] = [d for d in dirs if d not in ['__pycache__', '.ipynb_checkpoints']]
        for file in files:
            if not file.endswith(('.pyc', '.ipynb')):
                file_path = Path(root) / file
                arcname = file_path.relative_to(source_dir)
                print(f"Adding: {arcname}")
                zipf.write(file_path, arcname)

print(f"\nCreated: {zip_path}")

try:
    from google.colab import files
    files.download("demos/food_classifier.zip")
except:
    print("Not running in Google Colab, can't use google.colab.files.download(), please manually download.")

Adding: app.py
Adding: model.py
Adding: pretrained_effnetb2_feature_extractor_hamburger_hot_dog_french_fries.pth
Adding: requirements.txt
Adding: examples\1008163.jpg
Adding: examples\1033213.jpg
Adding: examples\10500.jpg

Created: demos\food_classifier.zip
Not running in Google Colab, can't use google.colab.files.download(), please manually download.


Woohoo!

Looks like our `zip` command was successful.

If you're running this notebook in Google Colab, you should see a file start to download in your browser.

### 9.2 Running our Food Classifier demo locally

If you download the `food_classifier.zip` file, you can test it locally by:
1. Unzipping the file.
2. Opening terminal or a command line prompt.
3. Changing into the `food_classifier` directory (`cd food_classifier`).
4. Creating an environment (`python3 -m venv env`).
5. Activating the environment (`source env/bin/activate`).
5. Installing the requirements (`pip install -r requirements.txt`, the "`-r`" is for recursive).
    * **Note:** This step may take 5-10 minutes depending on your internet connection. And if you're facing errors, you may need to upgrade `pip` first: `pip install --upgrade pip`.
6. Run the app (`python3 app.py`).

This should result in a Gradio demo just like the one we built above running locally on your machine at a URL such as `http://127.0.0.1:7860/`.


### 9.3 Uploading to Hugging Face

We've verified our Food Classifier app works locally, however, the fun of creating a machine learning demo is to show it to other people and allow them to use it.

To do so, we're going to upload our Food Classifier demo to Hugging Face. 

1. [Sign up](https://huggingface.co/join) for a Hugging Face account. 
2. Start a new Hugging Face Space by going to your profile and then [clicking "New Space"](https://huggingface.co/new-space).
    * **Note:** A Space in Hugging Face is also known as a "code repository" (a place to store your code/files) or "repo" for short.
3. Give the Space a name, for example, mine is called `poridhi001/food_classifier`, you can see it here: https://huggingface.co/spaces/poridhi001/food_classifier
4. Select a license (I used [MIT](https://opensource.org/licenses/MIT)).
5. Select Gradio as the Space SDK (software development kit). 
   * **Note:** You can use other options such as Streamlit but since our app is built with Gradio, we'll stick with that.
6. Choose whether your Space is it's public or private (I selected public since I'd like my Space to be available to others).
7. Click "Create Space".
8. Clone the repo locally by running something like: `git clone https://huggingface.co/spaces/[YOUR_USERNAME]/[YOUR_SPACE_NAME]` in terminal or command prompt.
    * **Note:** You can also add files via uploading them under the "Files and versions" tab.
9. Copy/move the contents of the downloaded `food_classifier` folder to the cloned repo folder.
10. To upload and track larger files (e.g. files over 10MB or in our case, our PyTorch model file) you'll need to [install Git LFS](https://git-lfs.github.com/) (which stands for "git large file storage").
11. After you've installed Git LFS, you can activate it by running `git lfs install`.
12. In the `food_classifier` directory, track the files over 10MB with Git LFS with `git lfs track "*.file_extension"`.
    * Track EffNetB2 PyTorch model file with `git lfs track "pretrained_effnetb2_feature_extractor_hamburger_hot_dog_french_fries.pth"`.
13. Track `.gitattributes` (automatically created when cloning from HuggingFace, this file will help ensure our larger files are tracked with Git LFS). You can see an example `.gitattributes` file on the [Food Classifier Hugging Face Space](https://huggingface.co/spaces/poridhi001/food_classifier/blob/main/.gitattributes).
    * `git add .gitattributes`
14. Add the rest of the `food_classifier` app files and commit them with: 
    * `git add *`
    * `git commit -m "first commit"`
15. Push (upload) the files to Hugging Face:
    * `git push`
16. Wait 3-5 minutes for the build to happen (future builds are faster) and your app to become live!

If everything worked, you should see a live running example of our Food Classifier Gradio demo like the one here: https://huggingface.co/spaces/poridhi001/food_classifier

![Deployed Food Classifier Demo on HuggingFace Spaces](https://github.com/poridhiEng/lab-asset/blob/main/tensorcode/Deep-learning-with-pytorch/Model-Deployment/Lab_02/images/img5.png?raw=true)

## Key Takeaways

- Gradio makes it easy to create interactive ML demos
- Hugging Face Spaces provides free hosting for ML applications
- Proper app structure includes: model.py, app.py, requirements.txt
- Deployed models can be shared and used by anyone with the URL

## Congratulations!

You've completed the PyTorch Model Deployment labs! You now know how to:
1. Create feature extractor models (EffNetB2 and ViT)
2. Compare models across multiple metrics
3. Build interactive Gradio demos
4. Deploy models to Hugging Face Spaces